In [31]:
from vanna.ollama import Ollama
from vanna.base import VannaBase
from pymilvus import MilvusClient, model
from vanna.milvus import Milvus_VectorStore
import ollama
import torch
import pyodbc

In [32]:
import requests
import numpy as np
from typing import List, Union


class CustomEmbeddingFunction:
    def __init__(self, api_url="http://10.13.18.40:11435", api_key=None, model_name="nomic-embed-text"):
        # Changed default to http instead of https, as Ollama typically runs on http
        self.api_url = api_url.rstrip('/')  # Remove trailing slash if present
        self.api_key = api_key
        self.model_name = model_name
        
        # Set up headers for API requests
        self.headers = {
            "Content-Type": "application/json"
        }
        
        # Add API key to headers if provided
        if self.api_key:
            self.headers["Authorization"] = f"Bearer {self.api_key}"
    
    def _get_embeddings(self, texts: List[str]) -> np.ndarray:
        embeddings_list = []
        
        # Ollama processes one text at a time, so we need to loop through texts
        for text in texts:
            # Correct endpoint for Ollama embeddings API
            endpoint = f"{self.api_url}/api/embeddings"
            
            # Correct payload format for Ollama
            payload = {
                "model": self.model_name,
                "prompt": text
            }
            
            response = requests.post(
                endpoint,
                headers=self.headers,
                json=payload,
                verify=False  # Added for self-signed certificates, remove if not needed
            )
            
            # Check for errors
            response.raise_for_status()
            
            # Parse response and extract embeddings
            result = response.json()
            embedding = np.array(result.get("embedding", []))
            embeddings_list.append(embedding)
        
        # Stack all embeddings into a single numpy array
        embeddings = np.stack(embeddings_list) if embeddings_list else np.array([])
        return embeddings
    
    def __call__(self, texts: Union[str, List[str]]) -> np.ndarray:
        if isinstance(texts, str):
            texts = [texts]
            
        return self._get_embeddings(texts)
    
    def encode_documents(self, texts: Union[str, List[str]]) -> np.ndarray:
        if isinstance(texts, str):
            texts = [texts]
            
        return self._get_embeddings(texts)
    
    def encode_queries(self, texts: Union[str, List[str]]) -> np.ndarray:
        if isinstance(texts, str):
            texts = [texts]
            
        return self._get_embeddings(texts)

In [33]:
milvus_uri = "http://10.13.18.40:19530"
milvus_client = MilvusClient(uri=milvus_uri,db_name='SQLRAG_SN_ALMS')#db_name='SQLRAG'


class MyVanna(Milvus_VectorStore, Ollama):
    def __init__(self, config=None):
        Milvus_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn_milvus = MyVanna(config={
                    'model': 'mistral-nemo',
                    'milvus_client': milvus_client,
                    "embedding_function": CustomEmbeddingFunction(),
                    "n_results": 3,
                    }
            )

In [45]:
db = 'YM_ALMS_TEST'

vn_milvus.connect_to_mssql(odbc_conn_str=f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER=10.22.66.37;DATABASE={db};UID=ALS2.0_Normal_readonly;PWD=6tfcXDR%;Trusted_Connection=no;Encrypt=yes;TrustServerCertificate=yes')

In [34]:
db = 'SN_ALMS'
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=UTCSNEDCLSNR02;'
    f'DATABASE={db};'
    'UID=ALS2.0_readonly;'
    'PWD=5rdxZSE$;'
    'Trusted_Connection=no;'
    'Encrypt=yes;'
    'TrustServerCertificate=yes;'
)
    # sqlite_path = "./my-database.sqlite"
cursor = conn.cursor()

In [35]:
db = 'SN_ALMS'
vn_milvus.connect_to_mssql(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=UTCSNEDCLSNR02;'
    f'DATABASE={db};'
    'UID=ALS2.0_readonly;'
    'PWD=5rdxZSE$;'
    'Trusted_Connection=no;'
    'Encrypt=yes;'
    'TrustServerCertificate=yes;'
)

In [36]:
df_information_schema = vn_milvus.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")
df_information_schema

TABLE_CATALOG TABLE_SCHEMA                      TABLE_NAME  \
0         SN_ALMS          dbo                  EAP_MQ_History   
1         SN_ALMS          dbo                  EAP_MQ_History   
2         SN_ALMS          dbo                  EAP_MQ_History   
3         SN_ALMS          dbo                  EAP_MQ_History   
4         SN_ALMS          dbo                  EAP_MQ_History   
..            ...          ...                             ...   
207       SN_ALMS          dbo  V_AFT_Main_subEquipment_ForRMS   
208       SN_ALMS          dbo  V_AFT_Main_subEquipment_ForRMS   
209       SN_ALMS          dbo  V_AFT_Main_subEquipment_ForRMS   
210       SN_ALMS          dbo  V_AFT_Main_subEquipment_ForRMS   
211       SN_ALMS          dbo  V_AFT_Main_subEquipment_ForRMS   

                      COLUMN_NAME  ORDINAL_POSITION COLUMN_DEFAULT  \
0                Transaction_Name                 1           None   
1                 Transaction_TID                 2           None   
2                Transaction_Type                 3           None   
3        Transaction_DateTimeSent                 4           None   
4    Transaction_DateTimeReceived                 5           None   
..                            ...               ...            ...   
207       SubEquipment_RMSAbility                12           None   
208         SubEquipment_DCSGroup                13           None   
209       SubEquipment_DCSAbility                14           None   
210          Last_Modify_DateTime                15           None   
211              Last_Modify_User                16           None   

    IS_NULLABLE  DATA_TYPE  CHARACTER_MAXIMUM_LENGTH  CHARACTER_OCTET_LENGTH  \
0            NO    varchar                      16.0                    16.0   
1            NO   nvarchar                       8.0                    16.0   
2            NO    varchar                      16.0                    16.0   
3            NO  datetime2                       NaN                     NaN   
4            NO  datetime2                       NaN                     NaN   
..          ...        ...                       ...                     ...   
207          NO        bit                       NaN                     NaN   
208         YES   nvarchar                      16.0                    32.0   
209          NO        bit                       NaN                     NaN   
210          NO  datetime2                       NaN                     NaN   
211          NO    varchar                       8.0                     8.0   

     ...  DATETIME_PRECISION  CHARACTER_SET_CATALOG  CHARACTER_SET_SCHEMA  \
0    ...                 NaN                   None                  None   
1    ...                 NaN                   None                  None   
2    ...                 NaN                   None                  None   
3    ...                 7.0                   None                  None   
4    ...                 7.0                   None                  None   
..   ...                 ...                    ...                   ...   
207  ...                 NaN                   None                  None   
208  ...                 NaN                   None                  None   
209  ...                 NaN                   None                  None   
210  ...                 7.0                   None                  None   
211  ...                 NaN                   None                  None   

     CHARACTER_SET_NAME COLLATION_CATALOG COLLATION_SCHEMA  \
0                 cp950              None             None   
1               UNICODE              None             None   
2                 cp950              None             None   
3                  None              None             None   
4                  None              None             None   
..                  ...               ...              ...   
207                None              None         

In [28]:
plan = vn_milvus.get_training_plan_generic(df_information_schema)
plan

Train on Information Schema: SN_ALMS.dbo EAP_MQ_History
Train on Information Schema: SN_ALMS.dbo EAP_ALM_History
Train on Information Schema: SN_ALMS.dbo AFT_Main_subEquipment
Train on Information Schema: SN_ALMS.dbo AFT_Main
Train on Information Schema: SN_ALMS.dbo ALM_Group
Train on Information Schema: SN_ALMS.dbo AFT_LoginHis
Train on Information Schema: SN_ALMS.dbo ALS_RequisitionRecord
Train on Information Schema: SN_ALMS.dbo AFT_InfoChange
Train on Information Schema: SN_ALMS.dbo ALS_FabController
Train on Information Schema: SN_ALMS.dbo ALM_Freq
Train on Information Schema: SN_ALMS.dbo ALM_ID_Detail
Train on Information Schema: SN_ALMS.dbo ALM_LoginHis
Train on Information Schema: SN_ALMS.dbo ALM_TeamplusHis
Train on Information Schema: SN_ALMS.dbo V_AFT_Main_subEquipment_ForRMS

In [29]:
vn_milvus.train(plan=plan)

In [30]:
training_data = vn_milvus.get_training_data()
training_data

id question  \
0   1e102456-3ac0-4762-9c0e-d885a648946a-doc     None   
1   23236df6-b85e-4d95-952f-f641410a0339-doc     None   
2   35be33c7-c14f-4339-940f-5923639656e8-doc     None   
3   4c87a82b-601e-421d-9eaf-e3b3ed98b063-doc     None   
4   4d98cbc6-eeec-4a0e-a173-6917b4fe032b-doc     None   
5   4e21e6eb-1a98-4fe0-8d09-55ae8582e735-doc     None   
6   7101adb3-8030-4f91-a2bf-8ebc8c33945e-doc     None   
7   79b4fb12-deb7-435d-a18e-db9187c54ad6-doc     None   
8   8833d74c-c741-4c7b-a8f9-c147b72e9a08-doc     None   
9   a49202f6-784f-4e76-af48-092367623841-doc     None   
10  a60b07ea-4bcd-4281-a7f0-41c50c5b5d6f-doc     None   
11  ccbd2cc8-0de3-401d-849d-517091dfb21e-doc     None   
12  d2fb658b-bdb5-4139-b5e2-02501e65a892-doc     None   
13  dcbf39a8-6887-45f5-b59a-1db908cf90a7-doc     None   

                                              content  
0   以下是SN_ALMS資料庫中EAP_MQ_History表格包含的欄位:\n\n|    |...  
1   以下是SN_ALMS資料庫中ALM_TeamplusHis表格包含的欄位:\n\n|    ...  
2   以下是SN_ALMS資料庫中ALS_RequisitionRecord表格包含的欄位:\n\...  
3   以下是SN_ALMS資料庫中V_AFT_Main_subEquipment_ForRMS表格...  
4   以下是SN_ALMS資料庫中AFT_Main_subEquipment表格包含的欄位:\n\...  
5   以下是SN_ALMS資料庫中ALM_LoginHis表格包含的欄位:\n\n|     | ...  
6   以下是SN_ALMS資料庫中ALS_FabController表格包含的欄位:\n\n|  ...  
7   以下是SN_ALMS資料庫中ALM_Freq表格包含的欄位:\n\n|     | TABL...  
8   以下是SN_ALMS資料庫中ALM_ID_Detail表格包含的欄位:\n\n|     |...  
9   以下是SN_ALMS資料庫中ALM_Group表格包含的欄位:\n\n|     | TAB...  
10  以下是SN_ALMS資料庫中AFT_InfoChange表格包含的欄位:\n\n|     ...  
11  以下是SN_ALMS資料庫中AFT_LoginHis表格包含的欄位:\n\n|     | ...  
12  以下是SN_ALMS資料庫中AFT_Main表格包含的欄位:\n\n|    | TABLE...  
13  以下是SN_ALMS資料庫中EAP_ALM_History表格包含的欄位:\n\n|    ...

In [17]:
ddl = """
CREATE TABLE [dbo].[EAP_MQ_History](
	[Transaction_Name] [varchar](16) NOT NULL,
	[Transaction_TID] [nvarchar](8) NOT NULL,
	[Transaction_Type] [varchar](16) NOT NULL,
	[Transaction_DateTimeSent] [datetime2](7) NOT NULL,
	[Transaction_DateTimeReceived] [datetime2](7) NOT NULL,
	[IsLog] [varchar](8) NOT NULL,
	[Source_IP] [varchar](16) NOT NULL,
	[Source_Queue] [varchar](32) NOT NULL,
	[Fab] [varchar](4) NULL,
	[MachineNo] [varchar](16) NOT NULL,
	[DateTime] [datetime2](7) NOT NULL,
	[Alarm_ID] [varchar](16) NOT NULL,
	[Alarm_Message] [varchar](256) NOT NULL,
	[LotNum] [varchar](16) NULL,
	[Alarm_CD] [tinyint] NULL,
	[Alarm_Status] [varchar](8) NOT NULL,
	[RecordTime] [datetime2](7) NOT NULL,
	[Status] [bit] NOT NULL,
	[MSMQ_XML] [xml] NOT NULL,
 CONSTRAINT [PK_EAP_MQ_History] PRIMARY KEY CLUSTERED 
(
	[RecordTime] DESC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
    """
vn_milvus.train(ddl=ddl)

Adding ddl: 
CREATE TABLE [dbo].[EAP_MQ_History](
	[Transaction_Name] [varchar](16) NOT NULL,
	[Transaction_TID] [nvarchar](8) NOT NULL,
	[Transaction_Type] [varchar](16) NOT NULL,
	[Transaction_DateTimeSent] [datetime2](7) NOT NULL,
	[Transaction_DateTimeReceived] [datetime2](7) NOT NULL,
	[IsLog] [varchar](8) NOT NULL,
	[Source_IP] [varchar](16) NOT NULL,
	[Source_Queue] [varchar](32) NOT NULL,
	[Fab] [varchar](4) NULL,
	[MachineNo] [varchar](16) NOT NULL,
	[DateTime] [datetime2](7) NOT NULL,
	[Alarm_ID] [varchar](16) NOT NULL,
	[Alarm_Message] [varchar](256) NOT NULL,
	[LotNum] [varchar](16) NULL,
	[Alarm_CD] [tinyint] NULL,
	[Alarm_Status] [varchar](8) NOT NULL,
	[RecordTime] [datetime2](7) NOT NULL,
	[Status] [bit] NOT NULL,
	[MSMQ_XML] [xml] NOT NULL,
 CONSTRAINT [PK_EAP_MQ_History] PRIMARY KEY CLUSTERED 
(
	[RecordTime] DESC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) 

'288166e8-7cd4-40d8-b235-f1d79f774239-ddl'

In [44]:
sql = vn_milvus.generate_sql("how many customers from new york city?")

SQL Prompt: [{'role': 'system', 'content': 'You are a T-SQL / Microsoft SQL Server expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE [dbo].[AFT_LoginHis](\n\t[Employee] [varchar](8) NOT NULL,\n\t[Name] [nvarchar](8) NOT NULL,\n\t[Sect] [nvarchar](128) NOT NULL,\n\t[LoginDateTime] [datetime2](7) NOT NULL,\n\t[IPAddress] [varchar](15) NOT NULL,\n\t[Action] [nvarchar](256) NOT NULL\n    ) ON [PRIMARY]\n    \n\n\n    CREATE TABLE [dbo].[Alarm_ID_Detail_NCN](\n\t[PEModule] [varchar](4) NULL,\n\t[EQP_ID] [varchar](16) NULL,\n\t[Fab] [varchar](8) NULL,\n\t[Alarm_Type] [varchar](4) NULL,\n\t[ALM_Group] [varchar](256) NULL,\n\t[Alarm_ID] [varchar](64) NULL,\n\t[Alarm_Name] [nvarchar](max) NULL,\n\t[Alarm_Description] [nvarchar](max) NULL,\n\t[Group] [varchar](16) NULL,\n\t[Priority] [varchar](4) NULL,\n\t[Enable] [varchar](2) NULL

In [37]:
vn_milvus.train(
    question="找出設備負責人為陳松青所負責的設備ID，並找出這些設備的子機台ID，最後統計這些子機台的異常數量",
    sql="""
    SELECT DISTINCT 
    B.SubEquipmet_ID,
    B.SubEquipmet_Name,
    A.EE_Owner_02,
    B.Equipment_ID,
    B.Equipmet_Name,
    COUNT(C.Transaction_DateTimeSent)
    FROM YM_ALMS_TEST.dbo.AFT_Main A WITH(NOLOCK)
    INNER JOIN YM_ALMS_TEST.dbo.AFT_Main_subEquipment B WITH(NOLOCK)
    ON A.EQP_ID = B.Equipment_ID
    INNER JOIN YM_ALMS_TEST.dbo.EAP_MQ_History C WITH(NOLOCK)
    ON B.SubEquipmet_ID = C.MachineNo
    WHERE (A.EE_Owner_01 LIKE '%陳松青%' OR A.EE_Owner_02 LIKE '%陳松青%')
    AND C.RecordTime > GETDATE()-30
    GROUP BY 
    B.SubEquipmet_ID,
    B.SubEquipmet_Name,
    A.EE_Owner_02,
    B.Equipment_ID,
    B.Equipmet_Name
    """
)

'824ee032-76ef-4f5a-ae78-1bc1760be2e7-sql'

In [22]:
vn_milvus.train(documentation="表AFT_Main為機台資訊維護總表，其中欄位Fab代表廠別，EQP_NO為機台編號，EQP_ID為機台ID，，EE_Owner_01和EE_Owner_02為設備負責人")

Adding documentation....


Batches: 100%|██████████| 1/1 [00:00<00:00, 67.61it/s]


'c6b91d82-d8eb-48d3-aeda-a86a96010913-doc'

In [21]:
vn_milvus.train(documentation="AFT_Main_subEquipment 是一個用於管理半導體廠主設備下附屬設備資料的管理表，由三個識別鍵（Fab晶圓廠代號、Equipment_ID主設備識別碼、SubEquipment_ID附屬設備識別碼）作為主要鍵值，記錄廠區編號(Fab_ID)、設備名稱(Equipment_Name)、設備群組(Equipment_Group)、附屬設備名稱(SubEquipment_Name)，並追蹤三大自動化系統（SubEquipment_ALS警報系統、SubEquipment_RMS遠端監控、SubEquipment_DCS分散式控制）的群組配置與功能啟用狀態，另外還包含了Recipe群組設定(SubEquipment_RecipeGroup)以及是否用於DCS檢查RMS的標記(SubEquipmet_IsUse_ForDCSCheckRMS)，最後記錄最後修改時間（Last_Modify_DateTime）與修改者（Last_Modify_User）")

Adding documentation....


Batches: 100%|██████████| 1/1 [00:00<00:00, 77.57it/s]


'fc62322a-77e9-4d53-8e00-ac1f92323786-doc'

In [19]:
import logging
from datetime import datetime
from typing import List

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'mssql_ddl_training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

def get_all_tables(cursor) -> List[str]:
    """
    Get all table names from the database
    """
    try:
        cursor.execute("""
            SELECT TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
        """)
        tables = [row[0] for row in cursor.fetchall()]
        logging.info(f"Found {len(tables)} tables in the database")
        return tables
    except Exception as e:
        logging.error(f"Failed to get tables: {str(e)}")
        raise

def get_table_ddl(cursor, table_name: str) -> str:
    """
    Get DDL for a specific table
    """
    try:
        # This query will get the CREATE TABLE statement
        cursor.execute(f"""
            SELECT OBJECT_DEFINITION (OBJECT_ID(N'[dbo].[{table_name}]'))
        """)
        ddl = cursor.fetchone()[0]
        logging.info(f"Successfully retrieved DDL for table: {table_name}")
        return ddl
    except Exception as e:
        logging.error(f"Failed to get DDL for table {table_name}: {str(e)}")
        raise

def main(cursor, vn_milvus):
    """
    Main function to process all tables
    """
    start_time = datetime.now()
    logging.info(f"Starting DDL extraction and training process for database")
    
    try:
        # Get all tables
        tables = get_all_tables(cursor)
        
        # Process each table
        for table_name in tables:
            try:
                # Get DDL for the table
                ddl = get_table_ddl(cursor, table_name)
                
                # Train the model with the DDL
                logging.info(f"Starting training for table: {table_name}")
                vn_milvus.train(ddl=ddl)
                logging.info(f"Successfully completed training for table: {table_name}")
                
            except Exception as e:
                logging.error(f"Error processing table {table_name}: {str(e)}")
                continue

    except Exception as e:
        logging.error(f"Main process error: {str(e)}")
    
    finally:
        end_time = datetime.now()
        duration = end_time - start_time
        logging.info(f"Process completed. Total duration: {duration}")

if __name__ == "__main__":
    # Using the existing cursor and vn_milvus from your code
    main(cursor, vn_milvus)

2025-02-07 06:42:16,625 - INFO - Starting DDL extraction and training process for database
2025-02-07 06:42:16,631 - INFO - Found 13 tables in the database
2025-02-07 06:42:16,636 - INFO - Successfully retrieved DDL for table: EAP_MQ_History
2025-02-07 06:42:16,636 - INFO - Starting training for table: EAP_MQ_History
2025-02-07 06:42:16,637 - INFO - Successfully completed training for table: EAP_MQ_History
2025-02-07 06:42:16,640 - INFO - Successfully retrieved DDL for table: EAP_ALM_History
2025-02-07 06:42:16,640 - INFO - Starting training for table: EAP_ALM_History
2025-02-07 06:42:16,641 - INFO - Successfully completed training for table: EAP_ALM_History
2025-02-07 06:42:16,645 - INFO - Successfully retrieved DDL for table: AFT_Main_subEquipment
2025-02-07 06:42:16,645 - INFO - Starting training for table: AFT_Main_subEquipment
2025-02-07 06:42:16,647 - INFO - Successfully completed training for table: AFT_Main_subEquipment
2025-02-07 06:42:16,651 - INFO - Successfully retrieved DD